In [ ]:
# Let's start mounting the Google Drive folder
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive

In [ ]:
# If you need to rebuild MRtrix3 from the code, execute from this cell...
!apt-get install libeigen3-dev libqt5opengl5-dev libqt5svg5-dev libfftw3-dev

In [ ]:
!git clone https://github.com/MRtrix3/mrtrix3.git

In [ ]:
%cd mrtrix3

In [ ]:
!chmod +x configure
!chmod +x build
!./configure -nogui

In [ ]:
!./build
# ...to this one! Once you have done that, you will not have to do these steps again

In [ ]:
import os
os.environ['PATH'] += ":/gdrive/MyDrive/mrtrix3/bin"
!chmod +x /gdrive/MyDrive/mrtrix3/bin/*

In [ ]:
# Installing dipy to fetch some sample data
%cd /root
!pip install dipy

In [ ]:
from dipy.data import fetch_sherbrooke_3shell

fetch_sherbrooke_3shell()

In [ ]:
# Time for some processing!
!dwi2response dhollander -fslgrad /root/.dipy/sherbrooke_3shell/HARDI193.bvec /root/.dipy/sherbrooke_3shell/HARDI193.bval /root/.dipy/sherbrooke_3shell/HARDI193.nii.gz wm.txt gm.txt csf.txt

In [ ]:
!dwi2fod msmt_csd -fslgrad /root/.dipy/sherbrooke_3shell/HARDI193.bvec /root/.dipy/sherbrooke_3shell/HARDI193.bval /root/.dipy/sherbrooke_3shell/HARDI193.nii.gz wm.txt wm.nii.gz gm.txt gm.nii.gz csf.txt csf.nii.gz

In [ ]:
%cd /gdrive/MyDrive

In [ ]:
# Let's grab mrpeek for the last bit
!git clone https://github.com/MRtrix3/mrpeek.git

In [ ]:
%cd mrpeek

In [ ]:
!../mrtrix3/build

In [ ]:
# Generate root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

# Download ngrok
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
# Setup sshd
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
# Set root password
!echo root:$password | chpasswd
!mkdir -p /var/run/sshd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
!echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

# Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

In [ ]:
# Create tunnel
!chmod +x ngrok
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
# Get public address
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('SSH to: root@', json.load(sys.stdin)['tunnels'][0]['public_url'][6:], sep='')"
# Print root password
print("Root password: {}".format(password))        

In [ ]:
# You can access now the colab instance from SSH, and look at your data using mrpeek!
# You just need a suitable terminal